In [1]:
!python -m venv venv

Could not find platform independent libraries <prefix>


In [2]:
!pip install torch torchaudio librosa numpy scikit-learn

^C


Could not find platform independent libraries <prefix>


In [67]:
import os
import random

def find_wav_files(base_dir, is_train:bool, per_class=100):
    sub_root = '1.Training' if is_train else '2.Validation'

    wav_files = []
    emotion_dir = os.path.join(
        base_dir, 
        '01.데이터', 
        sub_root,
        '원천데이터',
        ('T' if is_train else 'V') + 'S1',
        ('T' if is_train else 'V') + 'S1',
        '1.감정'
    )
    
    for emo in os.listdir(emotion_dir):
        emo_path = os.path.join(emotion_dir, emo)
        if not os.path.isdir(emo_path): continue
        for sub_folder in os.listdir(emo_path):
            sub_emo_path = os.path.join(emo_path, sub_folder)
            if not os.path.isdir(sub_emo_path): continue

            wav_list = [
                os.path.join(sub_emo_path, f)
                for f in os.listdir(sub_emo_path) if f.endswith('.wav')
            ]
            if len(wav_list) == 0:
                continue
            
            sampled = random.sample(wav_list, min(per_class, len(wav_list)))
            wav_files.extend(sampled)
    return wav_files


In [14]:
BASE_DIR = r'C:\Users\SSAFY\Downloads\015.감성 및 발화 스타일별 음성합성 데이터'
wav_list = find_wav_files(BASE_DIR, True)

In [15]:
len(wav_list)
wav_list[:5]

['C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\원천데이터\\TS1\\TS1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001047.wav',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\원천데이터\\TS1\\TS1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_000283.wav',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\원천데이터\\TS1\\TS1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001411.wav',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\원천데이터\\TS1\\TS1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001911.wav',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\원천데이터\\TS1\\TS1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001724.wav']

In [42]:
def match_label_file(wav_path, base_dir:str, is_train:bool):
    wav_tag = 'TS' if is_train else 'VS' 
    json_tag = 'TL' if is_train else 'VL' 
    label_path = wav_path.replace("원천데이터", "라벨링데이터")
    label_path = label_path.replace(f'{wav_tag}1', f'{json_tag}1')
    label_path = label_path.replace(".wav", ".json")
    return label_path

label_path_list = [match_label_file(wav, BASE_DIR, True) for wav in wav_list if os.path.exists(wav)]
label_path_list[:5]

['C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\TL1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001047.json',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\TL1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_000283.json',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\TL1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001411.json',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\TL1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001911.json',
 'C:\\Users\\SSAFY\\Downloads\\015.감성 및 발화 스타일별 음성합성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\TL1\\1.감정\\1.기쁨\\0001_G1A3E1S0C0_PSB\\0001_G1A3E1S0C0_PSB_001724.json']

In [43]:
import json
from pprint import pprint
def get_emotion_label(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # pprint(data['화자정보'])
    emotion = data['화자정보']['Emotion']
    return emotion

get_emotion_label(label_path_list[0])


'Happy'

In [44]:
temp = dict()
for label in label_path_list:
    emo = get_emotion_label(label)
    if temp.get(emo, False): temp[emo] += 1
    else: temp[emo] = 1

In [37]:
temp

{'Happy': 3300,
 'Sad': 3200,
 'Angry': 3200,
 'Anxious': 3200,
 'Hurt': 3200,
 'Embarrassed': 3200,
 'Neutrality': 3200}

In [40]:
_id = 0
label2id = dict()
id2label = dict()
for emo in temp.keys():
    label2id[emo] = _id
    id2label[_id] = emo
    _id += 1
    
del _id
print(label2id, id2label)

{'Happy': 0, 'Sad': 1, 'Angry': 2, 'Anxious': 3, 'Hurt': 4, 'Embarrassed': 5, 'Neutrality': 6} {0: 'Happy', 1: 'Sad', 2: 'Angry', 3: 'Anxious', 4: 'Hurt', 5: 'Embarrassed', 6: 'Neutrality'}


In [50]:
label_list = list(label2id[e] for e in map(get_emotion_label, label_path_list))
label_list[::3200]

[0, 0, 1, 2, 3, 4, 5, 6]

In [51]:
from sklearn.model_selection import train_test_split

train_files, test_files, train_labels, test_labels = train_test_split(
    wav_list, label_list, test_size=0.2, random_state=42)


In [6]:
import librosa
import numpy as np

def extract_mfcc(file_path, n_mfcc=40, max_len=300):
    y, sr = librosa.load(file_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0,0),(0,pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc


In [54]:
import torch
from torch.utils.data import Dataset

class AudioEmotionDataset(Dataset):
    def __init__(self, file_list, label_list):
        self.file_list = file_list
        self.label_list = label_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        mfcc = extract_mfcc(self.file_list[idx])
        label = self.label_list[idx]
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


from torch.utils.data import DataLoader

train_dataset = AudioEmotionDataset(train_files, train_labels)
test_dataset = AudioEmotionDataset(test_files, test_labels)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [58]:
import torch.nn as nn

class Emotion1DCNN(nn.Module):
    def __init__(self, n_mfcc=40, max_len=300, num_classes=7):
        super(Emotion1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(n_mfcc, 64, kernel_size=5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc1 = nn.Linear(128 * (max_len // 2), 128)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        # x: (batch, n_mfcc, max_len)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


In [59]:
torch.cuda.is_available()

True

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Emotion1DCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for mfccs, labels in train_loader:
        mfccs, labels = mfccs.to(device), labels.to(device)
        mfccs = mfccs  # (batch, n_mfcc, max_len)
        outputs = model(mfccs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1, Loss: 1.7221
Epoch 2, Loss: 1.4904
Epoch 3, Loss: 1.3824
Epoch 4, Loss: 1.3009
Epoch 5, Loss: 1.2176
Epoch 6, Loss: 1.1466
Epoch 7, Loss: 1.0700
Epoch 8, Loss: 1.0156
Epoch 9, Loss: 0.9515
Epoch 10, Loss: 0.8856
Epoch 11, Loss: 0.8362
Epoch 12, Loss: 0.7784
Epoch 13, Loss: 0.7198
Epoch 14, Loss: 0.6739
Epoch 15, Loss: 0.6250
Epoch 16, Loss: 0.5867
Epoch 17, Loss: 0.5386
Epoch 18, Loss: 0.5075
Epoch 19, Loss: 0.4711
Epoch 20, Loss: 0.4400


In [63]:
from sklearn.metrics import classification_report, confusion_matrix
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for x, y in test_loader:
        preds = model(x.to(device)).argmax(1).cpu().numpy()
        y_true.extend(y.numpy()); y_pred.extend(preds)
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred))


              precision    recall  f1-score   support

           0     0.7423    0.7934    0.7670       639
           1     0.6358    0.6916    0.6625       694
           2     0.4507    0.5306    0.4874       637
           3     0.3595    0.3895    0.3739       611
           4     0.3737    0.2900    0.3265       607
           5     0.4860    0.4356    0.4594       675
           6     0.6760    0.6091    0.6408       637

    accuracy                         0.5380      4500
   macro avg     0.5320    0.5343    0.5311      4500
weighted avg     0.5350    0.5380    0.5345      4500

[[507  18  82  14   3   7   8]
 [ 45 480  14  57  55  11  32]
 [ 66  33 338  31  22 103  44]
 [ 13  77  45 238  93 127  18]
 [ 17  87  53 151 176  53  70]
 [ 16  18 143 153  37 294  14]
 [ 19  42  75  18  85  10 388]]


In [142]:
def infer(model, audio_path, device):
    model.eval()
    mfcc = extract_mfcc(audio_path)                       # (n_mfcc, max_len)
    x = np.expand_dims(mfcc, 0)                  # (1, n_mfcc, max_len)
    x = torch.tensor(x, dtype=torch.float32)     # numpy → tensor
    x = x.to(device)                             # (1, C, L) on GPU/CPU

    with torch.no_grad():
        out = model(x)                           # (1, num_classes)
        pred = torch.argmax(out, dim=1).item()
    return id2label[pred]

In [209]:
val_list = find_wav_files(BASE_DIR, False, 1)
ans = {
    '기쁨': 'Happy',
    '슬픔': 'Sad',
    '분노': 'Angry',
    '당황': 'Embarrassed',
    '상처': 'Hurt',
    '중립': 'Neutrality',
    '불안': 'Anxious'
}

res = 0

for wav in val_list:
    emo = wav.split("1.감정\\")[1].split("\\")[0][2:]
    inferred = infer(model, wav, device)
    if inferred == ans[emo]: res += 1
res = res / len(val_list) * 100
print(f'{res}%')

28.57142857142857%


In [141]:
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score

# ... (모델 학습 및 평가 루프는 동일)

# 1. 예측값과 실제값 수집
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# 2. 주요 지표 출력
print("Accuracy:", accuracy_score(all_labels, all_preds))
print("Macro F1:", f1_score(all_labels, all_preds, average='macro'))
print("Macro Recall:", recall_score(all_labels, all_preds, average='macro'))
print("Macro Precision:", precision_score(all_labels, all_preds, average='macro'))

# 3. 전체 리포트 (클래스별 지표 포함)
print(classification_report(all_labels, all_preds, digits=4))


Accuracy: 0.538
Macro F1: 0.531078086798181
Macro Recall: 0.5342598295980111
Macro Precision: 0.5319771190610262
              precision    recall  f1-score   support

           0     0.7423    0.7934    0.7670       639
           1     0.6358    0.6916    0.6625       694
           2     0.4507    0.5306    0.4874       637
           3     0.3595    0.3895    0.3739       611
           4     0.3737    0.2900    0.3265       607
           5     0.4860    0.4356    0.4594       675
           6     0.6760    0.6091    0.6408       637

    accuracy                         0.5380      4500
   macro avg     0.5320    0.5343    0.5311      4500
weighted avg     0.5350    0.5380    0.5345      4500



In [1]:
torch.save(model.state_dict(), "cnn_1d_model.pth")

NameError: name 'torch' is not defined